In [176]:
import igraph as ig
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing, global_mean_pool

# Define the Apple Silicon device
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(device)

mps


In [177]:
## 1. 数据预处理
# 读取网络数据
edge_file = "../data/raw/european/powergridEU_E.csv"
coord_file = "../data/raw/european/powergridEU_V.csv"

# 读取数据
edges = pd.read_csv(edge_file, header=None).values.tolist()
coords = pd.read_csv(coord_file, header=None)  # 节点坐标数据

# 创建无向图
g = ig.Graph.TupleList(edges, directed=False)
g.vs["name"] = [int(v) for v in g.vs["name"]]  # 节点名转为整数

num_nodes, num_edges = g.vcount(), g.ecount()
print("Number of nodes:", num_nodes, "Number of edges:", num_edges)

Number of nodes: 13478 Number of edges: 16922


In [178]:
distances = np.load("../data/processed/distances.npy", allow_pickle=True)
print("distances:", distances.shape)
print("-" * 20)
adj_sparse = sp.load_npz("../data/processed/adj_sparse.npz")
print("邻接矩阵 adj_sparse:", adj_sparse.shape)
print("-" * 20)
B1 = sp.load_npz("../data/processed/B1.npz")
print("节点-边关联矩阵 B1:", B1.shape)
print("-" * 20)
L1_tilde = sp.load_npz("../data/processed/L1_tilde.npz")
L1_tilde = torch.tensor(L1_tilde.toarray(), dtype=torch.float32)  # 转换为密集矩阵
print("Hodge 1-Laplacian 矩阵 L1_tilde:", L1.shape)
print("-" * 20)
X = np.load("../data/processed/X.npy", allow_pickle=True)
print("X:", X.shape)

distances: (13478, 13478)
--------------------
邻接矩阵 adj_sparse: (13478, 13478)
--------------------
节点-边关联矩阵 B1: (13478, 16922)
--------------------
Hodge 1-Laplacian 矩阵 L1_tilde: torch.Size([16922, 16922])
--------------------
X: (13478, 5)


In [179]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HigherOrderSimplicialConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.theta = nn.Linear(in_channels, out_channels)  # 可训练权重 Θ_H
        self.bn = nn.BatchNorm1d(out_channels)

    def forward(self, Z_H, L1_tilde):
        # 公式: Z_H^{(ℓ+1)} = max(Ψ(˜L1 Z_H^{(ℓ)} Θ))
        # 1. 线性变换 Θ_H
        Z_theta = self.theta(Z_H)  # [M, out_channels]

        # 2. Hodge 1-Laplacian 矩阵乘法
        if L1_tilde.is_sparse:
            Z_conv = torch.sparse.mm(L1_tilde, Z_theta)
        else:
            Z_conv = torch.mm(L1_tilde, Z_theta)  # [M, out_channels]

        # 3. 非线性变换 Ψ (BatchNorm + ReLU)
        Z_psi = F.relu(self.bn(Z_conv))

        # 4. 元素级最大池化（沿特征维度取每个边的最大值）
        Z_max, _ = torch.max(Z_psi, dim=1, keepdim=True)  # [M, 1]

        return Z_max

class HoSC(nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super().__init__()
        self.layers = nn.ModuleList()
        current_dim = input_dim
        for dim in hidden_dims:
            self.layers.append(HigherOrderSimplicialConv(current_dim, dim))
            current_dim = 1  # 每层输出维度为1

    def forward(self, edge_attr, L1_tilde):
        Z_list = []
        Z_H = edge_attr
        for layer in self.layers:
            Z_H = layer(Z_H, L1_tilde)
            Z_list.append(Z_H)

        # 沿特征维度拼接所有层的输出
        return torch.cat(Z_list, dim=1)  # [M, num_layers]

class HOTNet(nn.Module):
    def __init__(self, edge_features, hidden_dims, num_classes=1):
        super().__init__()
        self.hosc = HoSC(edge_features, hidden_dims)
        self.fc = nn.Linear(len(hidden_dims), num_classes)

    def forward(self, data):
        # 提取边特征和Hodge矩阵
        edge_attr = data.edge_attr  # [M, edge_features]
        L1_tilde = data.L1_tilde    # [M, M]

        # 通过HoSC模块得到边级嵌入
        Z_H = self.hosc(edge_attr, L1_tilde)  # [M, num_layers]

        # 全局池化（对边特征进行全局平均）
        graph_embedding = Z_H.mean(dim=0).unsqueeze(0)  # [1, num_layers]

        # 分类头
        out = self.fc(graph_embedding)
        return torch.sigmoid(out)

In [135]:
## 数据预处理
# 模拟节点特征（例如电压、功率等）
node_features = torch.randn(num_nodes, 3)  # 3维节点特征
print(node_features.shape)

# 模拟边特征（例如电阻、电抗等）
edge_features = torch.randn(num_edges, 2)  # 2维边特征
print(edge_features.shape)

# 构建边索引，将edges转换为PyTorch张量
edges = pd.read_csv(edge_file, header=None).to_numpy()
edge_index = torch.tensor(edges, dtype=torch.long).T
print(edge_index.shape)

# 添加批次信息（假设只有一个图）
batch = torch.zeros(num_nodes, dtype=torch.long)
print(batch.shape)

torch.Size([13478, 3])
torch.Size([16922, 2])
torch.Size([2, 16922])
torch.Size([13478])


In [138]:
# 构建图数据对象
data = Data(
    x=node_features,  # 节点特征 [num_nodes, num_node_features]
    edge_index=edge_index,  # 边索引  torch.long [2, num_edges]
    edge_attr=edge_features,  # 边特征 [num_edges, num_edge_features]
    batch=batch,  # 批次信息 torch.long [num_nodes]
    L1=L1  # Hodge 1-Laplacian 矩阵 [num_edges, num_edges]
)
# 将数据移动到 Apple Silicon
data = data.to(device)
# 构建数据加载器
data_list = [data]
loader = DataLoader(data_list, batch_size=1, shuffle=False)

In [180]:
# 模型配置
model = HOTNet(edge_features=2, hidden_dims=[16, 32, 64])

# 前向传播
output = model(Data(edge_attr=edge_features, L1_tilde=L1_tilde))
print(output.shape)  # 输出形状: [1, 1]

torch.Size([1, 1])


In [183]:
output

tensor([[0.6597]], grad_fn=<SigmoidBackward0>)